# Our first operator in starting to use the software stack


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/GP211/2023-fall-class-notebooks/blob/main/in-class/Derivative.ipynb)

In [ ]:
%load_ext autoreload
%autoreload 2
import sys



In [ ]:
! pip install  "sep_plot @ git+https://github.com/SEP-software/sep-plot.git@6331a1e36d8e3cdb4cfbc3539f31bdad1eb465a7" 

In [ ]:
! wget https://github.com/GP211/2023-fall-class-notebooks/raw/main/data/bay.H

In [ ]:
import matplotlib.pyplot as plt
from sep_python import default_io
vec=default_io.vector_from_storage("./bay.H")
plt.imshow(vec.get_nd_array())
v=vec.get_nd_array()


In [ ]:
from abc import ABC, abstractmethod
class operator(ABC):
  """An abstract class for operators for inverse problems"""
  def __init__(self,domain,range):
    """Default initializer for an abstract class

      domain - Domain of operator
      range. - Range of operator

    """
    self._domain=domain.clone()
    self._range=range.clone()

  @abstractmethod
  def forward(self,add,model,data):
    """
      Run a forward

      add - Whether or not to add to the output
      model - Model (input)
      data. - Data (output)


    """


  @abstractmethod
  def adjoint(self,add,model,data):
    """
      Run an adjoint

      add - Whether or not to add to the output
      model - Model (output)
      data. - Data (output)


    """

  def check_same(self,model,data):
    """
      Check to see if model and data match operator initialization

        model - Model space
        data. - data space
    """
    if not self._domain.check_same(model):
      raise Exception("model and domain don't match")

    if not self._range.check_same(data):
      raise Exception("range and data don't match")


In [ ]:
class deriv_op(operator):

    def __init__(self,model,data):
        super().__init__(model,data)

    def forward(self,add,model,data):
        self.check_same(model,data)
        if not add: 
            data.zero()
        m=model.get_nd_array()
        d=data.get_nd_array()
        for i in range(m.shape[0]-1):
            for j in range(m.shape[1]):
                d[i+1,j]+=m[i+1,j]-m[i,j]
    def adjoint(self,add,model,data):
        self.check_same(model,data)
        if not add: 
            model.zero()
        m=model.get_nd_array()
        d=data.get_nd_array()

        for i in range(m.shape[0]):
            for j in range(m.shape[1]-1):
                m[i,j+1]+=d[i,j+1]
                m[i,j]-=d[i,j+1]
                

In [ ]:
mod=vec.clone()
dat=vec.clone()
dat.zero()
op=deriv_op(mod,dat)

In [ ]:
import sep_plot
op.forward(False,mod,dat)
sep_plot.Grey(dat)

## Method 2 write 1-D vector operation

In [ ]:
def forward(self,add,model,data):
    self.check_same(model,data)
    if not add: 
        data.zero()
    m=model.get_nd_array()
    d=data.get_nd_array()

    for j in range(m.shape[1]-1):
        d[:,j+1]+=m[:,j+1]-m[:,j]

## Method 3 write complete numpy vector operation

In [ ]:
def forward(self,add,model,data):
    self.check_same(model,data)
    if not add: 
        data.zero()
    m=model.get_nd_array()
    d=data.get_nd_array()

    d[:,1:]+=m[:,1:]-m[:,:-1]


## Methods 4 write in terms of numpy

In [ ]:

@numba.njit(parallel=True)
def forward_deriv(mod,dat):
    for i in numba.prange(m.shape[0]):
        for j in range(m.shape[1]-1):
            d[i,j+1]+=m[i,j+1]-m[i,j]


In [ ]:
import sep_plot

In [ ]:
sep_plot.Grey(vec)

In [ ]:
sep_plot.Grey(vec)